# Descrição:

O objetivo desse trabalho é extrair dados de imóveis postados no site www.dfimoveis.com.br e construir um modelo de estimação em cima desses dados com a técnica de Regressão Linear.

Informações da base de dados:

<ul style='font-size: 18px; line-height: 2; text-align: justify;'>
    <li><b>preco</b> - Preço do Imóvel (R$)</li>
    <li><b>area</b> - Área total do Imóvel (m)</li>
    <li><b>tipo</b> - Tipo do Imóvel</li>
    <li><b>local</b> - Localização real do Imóvel</li>
    <li><b>quartos</b> - Quantidade de quartos do Imóvel</li>
</ul>



# Etapa de extração de dados do site DFImoveis com BeautifulSoup

In [1]:
import bs4
import urllib.request as urllib_request
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.error import URLError, HTTPError
from urllib.request import urlopen

In [ ]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

In [ ]:
uf = input("digite a uf de interesse:").lower() # df

digite a uf de interesse:df


In [ ]:
cidade = input("digite a cidade de interesse:") # guara

digite a cidade de interesse:guara


In [ ]:
cidade = cidade.replace(' ', '-')

url = 'https://www.dfimoveis.com.br/aluguel/'+str(uf)+'/'+str(cidade)+'/imoveis/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
req = Request(url, headers = headers)
response = urlopen(req)
html = trata_html(response.read().decode('utf-8'))
    
html

'<!DOCTYPE html><html lang="pt-BR"><head><script> (function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ \'gtm.start\': new Date().getTime(), event: \'gtm.js\' }); var f = d.getElementsByTagName(s)[0], j = d.createElement(s), dl = l != \'dataLayer\' ? \'&l=\' + l : \'\'; j.async = true; j.src = \'https://www.googletagmanager.com/gtm.js?id=\' + i + dl; f.parentNode.insertBefore(j, f); })(window, document, \'script\', \'dataLayer\', \'GTM-54QGRMD\'); </script><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="ie=edge"><title> 423 Imóveis para alugar no Guará, DF - DFimoveis.com</title><meta name="author" content="dfimoveis.com"><meta name="description" content="Aluguel de 423 Imóveis no Guará - DF nos bairros Guara II, Guara I, Quadras Economicas Lucio Costa. Encontre Imóveis para alugar no Guará - DF."><meta name="keywords" content="aluguel,df,guara,"><link rel="canonical" href="https://www.dfimovei

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

### Encontrando o número de páginas

In [ ]:
anuncios = soup.findAll('div', {'class': 'property js-propriedade'})

In [ ]:
cards = []

for anuncio in anuncios:
    card = {}
    
    card['preco'] = int(anuncio.find('span', {'class':'price'}).getText().split(',')[0].replace('.', ''))
    
    card['tipo'] = anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[0]
    
    card['area'] = int(anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[-2])
    
    card['local'] = anuncio.find('h3', {'class': 'property__title hide-mobile'}).find('a').getText()
    cards.append(card) 

###Extraindo os dados e salvando

In [ ]:
max_quartos = 4
cards = []

for i in range(max_quartos):
    url = 'https://www.dfimoveis.com.br/aluguel/'+uf+'/'+cidade+'/imoveis/'+str(i + 1)+'-quarto'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = trata_html(response.read().decode('utf-8'))
    soup = BeautifulSoup(html, 'html.parser')
    anuncios = soup.findAll('div', {'class': 'property js-propriedade'})
    
    total_anuncios = int(soup.find('aside').find('a', class_ = 'filter-card__link').findAll('div')[-1].getText().replace('(', '').replace(')', ''))
    qtd_paginas = round((total_anuncios/40)-1)
    
    for j in range(qtd_paginas):
        
        url = 'https://www.dfimoveis.com.br/aluguel/'+uf+'/'+cidade+'/imoveis/'+str(i + 1)+'-quarto?pagina='+str(j+1)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = trata_html(response.read().decode('utf-8'))
        soup = BeautifulSoup(html, 'html.parser')
        anuncios = soup.findAll('div', {'class': 'property js-propriedade'})
        
        for anuncio in anuncios:
            card = {}
            card['preco'] = int(anuncio.find('span', {'class':'price'}).getText().split(',')[0].replace('.', ''))
            card['area'] = anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[-2]
            card['tipo'] = anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[1]
            card['local'] = anuncio.find('h3', {'class': 'property__title hide-mobile'}).find('a').getText()
            
            cards.append(card) 
            
            
        
dataset = pd.DataFrame(cards)
    
dataset.to_csv('./dados-dfimoveis-'+str(cidade)+'.csv', sep = ';', index = False, encoding = 'utf-8')
dataset        

,preco,area,tipo,local
0,1500,34,Apartamento,"CRS 516 Bloco C, ASA SUL, BRASILIA"
1,2000,40,Apartamento,"SQNW 310, NOROESTE, BRASILIA"
2,2100,33,Apartamento,"Avenida das Araucárias, SUL, AGUAS CLARAS"
3,1300,26,Kitnet,"SGCV Lote 11, PARK SUL, BRASILIA"
4,2900,40,Hotel,"SHN Quadra 05 Bloco I, ASA NORTE, BRASILIA"
...,...,...,...,...
2155,3900,145,Apartamento,"Rua 8, NORTE, AGUAS CLARAS"
2156,21000,547,Casa,"SHIS QL 24 Conjunto 8, LAGO SUL, BRASILIA"
2157,14000,600,Casa,"Sítio Forquilha Rua 5, JARDIM BOTANICO, BRASI..."
2158,25000,690,Casa,"SHIS QI 19 Conjunto 13, LAGO SUL, BRASILIA"


In [ ]:
max_quartos = 4
cards = []

for i in range(max_quartos):
    url = 'https://www.dfimoveis.com.br/aluguel/'+uf+'/'+cidade+'/imoveis/'+str(i + 1)+'-quarto'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = trata_html(response.read().decode('utf-8'))
    soup = BeautifulSoup(html, 'html.parser')
    anuncios = soup.findAll('div', {'class': 'property js-propriedade'})
    
    anuncios_por_pagina = len(soup.find('ul', {'class': 'property-list'}).findAll('li', {'class': 'property-list__item'}))
    total_anuncios = int(soup.find('aside').find('a', class_ = 'filter-card__link').findAll('div')[-1].getText().replace('(', '').replace(')', ''))
    qtd_paginas = round((total_anuncios/anuncios_por_pagina))
    
    for j in range(qtd_paginas):
        
        url = 'https://www.dfimoveis.com.br/aluguel/'+uf+'/'+cidade+'/imoveis/'+str(i + 1)+'-quarto?pagina='+str(j+1)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = trata_html(response.read().decode('utf-8'))
        soup = BeautifulSoup(html, 'html.parser')
        anuncios = soup.findAll('div', {'class': 'property js-propriedade'})
        
        for anuncio in anuncios:
            card = {}
            card['preco'] = int(anuncio.find('span', {'class':'price'}).getText().split(',')[0].replace('.', ''))
            card['area'] = anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[-2]
            card['tipo'] = anuncio.find('li', {'class': 'hide-mobile'}).getText().split()[1]
            card['local'] = anuncio.find('h3', {'class': 'property__title hide-mobile'}).find('a').getText()
            card['quartos'] = int(anuncio.find('li', class_='hide-mobile').find_next_sibling().getText().split()[0])
            
            cards.append(card) 
            
            
        
dataset = pd.DataFrame(cards)
    
dataset.to_csv('./dados-dfimoveis-'+ str(cidade) +'com-quartos.csv', sep = ';', index = False, encoding = 'utf-8')
dataset        

,preco,area,tipo,local,quartos
0,1400,37,Apartamento,"EPTG QE 4, QUADRAS ECONOMICAS LUCIO COSTA, GU...",1
1,1300,33,Apartamento,"EPTG QE 2, QUADRAS ECONOMICAS LUCIO COSTA, GU...",1
2,1350,36,Apartamento,"Área Especial 02, GUARA II, GUARA",1
3,1400,38,Apartamento,"QE 1, GUARA I, GUARA",1
4,1500,56,Apartamento,"QE 38 Bloco E, GUARA II, GUARA",1
...,...,...,...,...,...
232,4300,400,Casa,"QE 17 Conjunto I, GUARA II, GUARA",5
233,2000,90,Casa,"QI 14, GUARA I, GUARA",4
234,2800,120,Casa,"QE 38 Conjunto O, GUARA II, GUARA",4
235,5400,430,Casa,"QE 20, GUARA I, GUARA",5


#Realizando Regressão Linear nos dados obtidos

In [2]:
!pip install category_encoders
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders.target_encoder import TargetEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.4 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
dados = pd.read_excel('dados-guara.xlsx')

In [ ]:
y = dados['preco']

In [ ]:
enc = TargetEncoder(cols=["tipo"])
enc.fit(dados, dados["preco"])
dados_encodados = enc.transform(dados)
dados_encodados

In [ ]:
X = dados_encodados[['area', 'quartos', "tipo"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2811)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
modelo = LinearRegression()

In [ ]:
modelo.fit(X_train, y_train)

In [ ]:
print('R² = {}'.format(modelo.score(X_train, y_train).round(2)))

In [ ]:
y_previsto = modelo.predict(X_test)

In [ ]:
area = 120
quartos = 3
tipo = 874.288221
entrada= [[area, quartos, tipo]]

print('{0:.2f} reais'.format(modelo.predict(entrada)[0]))